<a href="https://colab.research.google.com/github/MahmoodAbdali79/Face-ani-spoofing/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://keras.io/examples/vision/video_classification/
# https://keras.io/api/preprocessing/image/
# https://discuss.pytorch.org/t/how-to-go-about-unbalanced-data-in-a-binary-classification-in-pytorch/90655

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/shenasa/benchmark

/content/drive/MyDrive/shenasa/benchmark


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.preprocessing.image import img_to_array
from rPPG.rPPG_Extracter import *
from rPPG.rPPG_lukas_Extracter import *
from keras.models import model_from_json
import tensorflow as tf
from random import seed
# from random import randint
from random import sample

seed(1)

In [ ]:
dim = (128,128)

In [ ]:
cascPath = './cv2_model/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

In [ ]:
# extract frames from videos (each 5 frames)
# numberfolder_filemname_x,y,w,h_label.jpg

def extract_frames(path, max_frames=6):
  folders = os.listdir(path)
  os.mkdir(f"{path}/frames")
  
  for folder in folders:
    print(f'exracting foder {folder} ...')
    videos = os.listdir(f'{path}/{folder}')
    for video in videos:
      print(video.split('.')[0], ' ...')
      name = video.split('.')[0]
      if name in ['1','2','HR_1'] : label = '1'
      else : label = '0'
      cap = cv2.VideoCapture(f'{path}/{folder}/{video}')
      
      try:
        i = 0 
        counter = 0
        while True:
          ret, frame = cap.read()
          if not ret: break

          i += 1
          # detect face 
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)


          if i%5 == 0 and len(faces) != 0:
            x, y, w, h = faces[0]
            print( faces[0],len(faces[0]) , f'counter: {counter+1}')
            cv2.imwrite(f'{path}/frames/{folder}_{name}_{x},{y},{w},{h}_{label}.jpg', frame)
            counter += 1
          # if counter == max_frames: break
      finally:
        cap.release()

In [ ]:
extract_frames('test_release')

In [ ]:
def get_rppg_pred(frame):
    use_classifier = True  
                                 
    sub_roi = []           
    use_resampling = False  
    
    fs = 20

    timestamps = []
    time_start = [0]

    break_ = False

    rPPG_extracter = rPPG_Extracter()
    rPPG_Lukas_Extracter()
    bpm = 0
    
    dt = time.time()-time_start[0]
    time_start[0] = time.time()
    if len(timestamps) == 0:
        timestamps.append(0)
    else:
        timestamps.append(timestamps[-1] + dt)
        

    rPPG_extracter.measure_rPPG(frame,use_classifier,sub_roi) 
    rPPG = np.transpose(rPPG_extracter.rPPG)
    
        # Extract Pulse
    if rPPG.shape[1] > 10:
        if use_resampling :
            t = np.arange(0,timestamps[-1],1/fs)
            
            rPPG_resampled= np.zeros((3,t.shape[0]))
            for col in [0,1,2]:
                rPPG_resampled[col] = np.interp(t,timestamps,rPPG[col])
            rPPG = rPPG_resampled
        num_frames = rPPG.shape[1]

        
    return rPPG

In [ ]:
def make_pred(li):
    [single_img,rppg] = li
    # print(single_img.shape)
    single_img = cv2.resize(single_img, dim)
    single_x = img_to_array(single_img)
    single_x = np.expand_dims(single_x, axis=0)

    # print(single_img.shape)
    # print('============')
    # print(single_x.shape)
    # print('============')
    # print(rppg.shape)

    single_pred = model.predict([single_x,rppg])
    return single_pred

In [ ]:
def prediction_test(path):
  result = []
  label = []
  ma_frames = os.listdir(f'{path}/frames')
  counter = 0
  for i in ma_frames:
    print(f'{counter}:', i)
    im = cv2.imread(f'{path}/frames/{i}')
    if 'HR' in i:
      face = list(map(int, i.split('_')[3].split(',')))
    else:
      face = list(map(int, i.split('_')[2].split(',')))

    x, y, w, h = face
    sub_img=im[y:y+h,x:x+w]
    
    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T
    # print(sub_img.shape)
    # print(rppg_s)
    pred = make_pred([sub_img,rppg_s])
    result.append(pred)
    label.append(int(i.split('_')[-1].split('.')[0]))
    counter += 1

  result = np.array(result).reshape(len(result), -1) 
  result = tf.math.argmin(result, axis=1)
  label = tf.convert_to_tensor(label)

  return result, label
  

In [ ]:
json_file = open('../RGB_rPPG_merge_softmax_.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("../RGB_rPPG_merge_softmax_.h5")

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
result, label = prediction_test('test_release')

0: 15_7_170,195,226,226_0.jpg
1: 15_7_170,197,219,219_0.jpg
2: 15_7_166,196,224,224_0.jpg
3: 15_7_167,197,222,222_0.jpg
4: 15_7_165,195,226,226_0.jpg
5: 15_7_165,195,228,228_0.jpg
6: 15_7_166,197,223,223_0.jpg
7: 15_7_167,197,223,223_0.jpg
8: 15_7_166,198,224,224_0.jpg
9: 15_7_167,197,225,225_0.jpg
10: 15_7_166,197,224,224_0.jpg
11: 15_7_166,197,225,225_0.jpg
12: 15_7_167,197,224,224_0.jpg
13: 15_7_168,198,225,225_0.jpg
14: 15_7_167,198,227,227_0.jpg
15: 15_7_168,199,226,226_0.jpg
16: 15_7_166,198,226,226_0.jpg
17: 15_7_166,198,225,225_0.jpg
18: 15_7_167,199,224,224_0.jpg
19: 15_7_167,200,223,223_0.jpg
20: 15_8_183,201,182,182_0.jpg
21: 15_8_183,201,184,184_0.jpg
22: 15_8_182,201,184,184_0.jpg
23: 15_8_182,200,182,182_0.jpg
24: 15_8_183,202,182,182_0.jpg
25: 15_8_185,202,181,181_0.jpg
26: 15_8_184,203,184,184_0.jpg
27: 15_8_185,205,181,181_0.jpg
28: 15_8_185,204,183,183_0.jpg
29: 15_8_183,203,184,184_0.jpg
30: 15_8_184,203,182,182_0.jpg
31: 15_8_184,203,183,183_0.jpg
32: 15_8_185,204,1

/content/drive/My Drive/shenasa/benchmark/rPPG/rPPG_Extracter.py:24: RuntimeWarning: invalid value encountered in ulong_scalars
  r_avg = np.sum(frame[:,:,0])/num_pixels
/content/drive/My Drive/shenasa/benchmark/rPPG/rPPG_Extracter.py:25: RuntimeWarning: invalid value encountered in ulong_scalars
  g_avg = np.sum(frame[:,:,1])/num_pixels
/content/drive/My Drive/shenasa/benchmark/rPPG/rPPG_Extracter.py:26: RuntimeWarning: invalid value encountered in ulong_scalars
  b_avg = np.sum(frame[:,:,2])/num_pixels


Streaming output truncated to the last 5000 lines.
3950: 30_3_134,147,217,217_0.jpg
3951: 30_3_137,150,215,215_0.jpg
3952: 30_3_140,152,209,209_0.jpg
3953: 30_3_138,152,213,213_0.jpg
3954: 30_3_135,149,218,218_0.jpg
3955: 30_3_138,149,219,219_0.jpg
3956: 30_3_133,147,219,219_0.jpg
3957: 30_3_134,146,218,218_0.jpg
3958: 30_3_131,143,223,223_0.jpg
3959: 30_3_134,146,219,219_0.jpg
3960: 30_3_130,148,228,228_0.jpg
3961: 30_3_137,155,221,221_0.jpg
3962: 30_3_142,155,221,221_0.jpg
3963: 30_3_145,163,211,211_0.jpg
3964: 30_3_140,160,221,221_0.jpg
3965: 30_3_134,153,222,222_0.jpg
3966: 30_3_132,148,231,231_0.jpg
3967: 30_3_132,150,232,232_0.jpg
3968: 30_3_128,152,235,235_0.jpg
3969: 30_3_136,157,225,225_0.jpg
3970: 30_3_132,147,236,236_0.jpg
3971: 30_3_133,152,232,232_0.jpg
3972: 30_3_133,161,233,233_0.jpg
3973: 30_3_132,158,236,236_0.jpg
3974: 30_3_132,153,228,228_0.jpg
3975: 30_3_133,151,232,232_0.jpg
3976: 30_3_147,159,216,216_0.jpg
3977: 30_3_143,149,223,223_0.jpg
3978: 30_3_148,154,214,21

In [ ]:
len(os.listdir('test_release/frames'))

8950

In [ ]:
label.shape

TensorShape([8950])

In [ ]:
res = tf.math.confusion_matrix(label,result)
print(res)

tf.Tensor(
[[4829 1364]
 [1180 1577]], shape=(2, 2), dtype=int32)


In [ ]:
# test_30_180_1  first method : 0.29465575349061146
# (1083+141)/(2*2077)

# test_30_180_1 second method : 0.29809955540057537
# (6087+752)/(2*11471)

# test_30_180_1 third method : 0.29
# (4439+752)/(2*8950)

0.29

In [ ]:
# balance_30_300_7 first method : 0.36398651901781415
# (1504+8)/(2*2077)

# balance_30_300_7 second method : 0.3687124051957109
# (8346+113)/(2*11471)

# balance_30_300_7 third method : 0.26261877778746406
# (5912+113)/(2*11471)

0.26261877778746406

In [ ]:
# RGB_rPPG_merge_softmax_   first method : 0.343042850264805
# (1129+296)/(2*2077) 

# RGB_rPPG_merge_softmax_   second method : 0.3496643710225787
# (6445+1577)/(2*11471)

# RGB_rPPG_merge_softmax_   third method : 0.27922587394298665
# (4829+1577)/(2*11471)

In [ ]:
(2077)/(2*2077)

0.5

In [ ]:
(0)/(2*2077)

0.0

In [ ]:
files = os.listdir('test_release/frames')

In [ ]:
a = files[100]
print(a)

if 'HR' in a: print('_'.join(a.split('_')[1:3]))
else: print('_'.join(a.split('_')[1]))

15_HR_2_37,292,635,635_0.jpg
HR_2


In [ ]:
for f in files:
  if 'HR' in f: name = '_'.join(f.split('_')[1:3])
  else: name = '_'.join(f.split('_')[1])

  if name in ['5', '6', 'HR_3']:
    try:
      # print(f)
      os.remove('test_release/frames/'+f)
    except:pass

In [ ]:
len( os.listdir('test_release/frames'))

8950

In [ ]:
try:
  os.remove('test_release/frames/'+files[10])
except:pass

In [ ]:
def extract_frame(in_paht,  out_path):  #test_release/18/3.avi
  #extrame frae from a path and release in a other path
  folder = in_paht.split('/')[1] 
  name = in_paht.split('.')[0].split('/')[-1]
  label = '0'
  number_frame = 10 
  if name in ['1','2','HR_1'] : 
    label = '1'
    number_frame = 30

  print(f'    Extracting {name}.avi ...')

  cap = cv2.VideoCapture(in_paht)
  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  selected_frame = np.sort(sample(range(length), number_frame))
  last_frame = selected_frame[-1]

  print(selected_frame, len(selected_frame))

  try:
    i = 0 
    while True:
      ret, frame = cap.read()
      if not ret: break

      # detect face 
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

      if i in selected_frame and len(faces) != 0:
        print(f'in frame : {i}')
        x, y, w, h = faces[0]
        cv2.imwrite(f'{out_path}/{folder}_{name}_{x},{y},{w},{h}_{label}.jpg', frame)
    
      i += 1
      if i == last_frame+1:
        break

  finally:
    cap.release()


In [ ]:
extract_frame('test_release/18/1.avi', 'benchmark_type/real')

    Extracting 1.avi ...
[ 0  1  3  6  7  8 10 13 16 17 18 19 20 21 23 24 25 27 28 29 30 31 33 35
 36 41 42 45 46 47] 30
in frame : 0
in frame : 1
in frame : 3
in frame : 6
in frame : 7
in frame : 8
in frame : 10
in frame : 13
in frame : 16
in frame : 17
in frame : 18
in frame : 19
in frame : 20
in frame : 21
in frame : 23
in frame : 24
in frame : 25
in frame : 27
in frame : 28
in frame : 29
in frame : 30
in frame : 31
in frame : 33
in frame : 35
in frame : 36
in frame : 41
in frame : 42
in frame : 45
in frame : 46
in frame : 47


In [ ]:
!ls benchmark_type/real | wc -l

7


In [ ]:
len(os.listdir('benchmark_type/real'))

7

In [ ]:
def seprate_extract_frame(path):
  folders = os.listdir(path)
  
  for folder in folders:
    print(f'exracting foder {folder} ...')
    videos = os.listdir(f'{path}/{folder}')
    for video in videos:
      name = video.split('.')[0]
      # print(name, ' ...')
      print(f'{path}/{folder}/{video}')
      extract_frame(f'{path}/{folder}/{video}', out_path)
    
    break


In [ ]:
seprate_extract_frame('test_release')

exracting foder 18 ...
test_release/18/1.avi
test_release/18/2.avi
test_release/18/3.avi
test_release/18/4.avi
test_release/18/5.avi
test_release/18/6.avi
test_release/18/7.avi
test_release/18/8.avi
test_release/18/HR_1.avi
test_release/18/HR_2.avi
test_release/18/HR_3.avi
test_release/18/HR_4.avi


In [ ]:
# extract frames from videos (each 5 frames)
# numberfolder_filemname_x,y,w,h_label.jpg

def extract_frames(path, max_frames=6):
  folders = os.listdir(path)
  os.mkdir(f"{path}/frames")
  
  for folder in folders:
    print(f'exracting foder {folder} ...')
    videos = os.listdir(f'{path}/{folder}')
    for video in videos:
      print(video.split('.')[0], ' ...')
      name = video.split('.')[0]
      if name in ['1','2','HR_1'] : label = '1'
      else : label = '0'
      cap = cv2.VideoCapture(f'{path}/{folder}/{video}')
      
      try:
        i = 0 
        counter = 0
        while True:
          ret, frame = cap.read()
          if not ret: break

          i += 1
          # detect face 
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)


          if i%5 == 0 and len(faces) != 0:
            x, y, w, h = faces[0]
            print( faces[0],len(faces[0]) , f'counter: {counter+1}')
            cv2.imwrite(f'{path}/frames/{folder}_{name}_{x},{y},{w},{h}_{label}.jpg', frame)
            counter += 1
          # if counter == max_frames: break
      finally:
        cap.release()